<a href="https://colab.research.google.com/github/sharadv99/capstone-OpioidAddictionRisk/blob/master/Opioid_Project_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opioid Addiction Project
Python Notebook 1 of x

This notebook is just to initially explore the data.

### W210, Capstone
Summer 2019

Team:  Cameron Kennedy, Aditi Khullar, Rachel Kramer, Sharad Varadarajan

# 0. Load Libraries and Set Global Variables
This analysis is performed in the cells below.

In [0]:
#Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Set initial parameter(s)
pd.set_option('display.max_rows', 200)
pd.options.display.max_columns = 20
dataDir = './data/'

# 1. Load Data

In [0]:
#Load Data
dfNSDUHRaw = pd.read_csv(dataDir+'NSDUH_2017_Tab.tsv', sep='\t', low_memory=False)
#low_memory=False prevents an error

Note, I checked this against the .Rdata file and they appear to be the same.  `NA` values in R equate to `NaN` values here.

In [0]:
#Quick inspection
#dfRaw.head(20)

In [0]:
#dfNSDUHRaw.dtypes

In [0]:
#Write raw data to pkl file
#dfNSDUHRaw.to_pickle(dataDir + 'dfNSDUHRaw.pkl.zip', compression='zip')
'''This step is complete and there's no need to keep rewriting this file,
hence it's commented out
'''
;

''

# 2. Preprocessing

## 2.1 Remove Non Pain Reliever Users From the Data

This section will remove non-opioid users.

A reasonable approximation for additiction is misues, and a good candidate field for misuse is PNRNMREC: MOST RECENT PAIN RELIEVER MISUSE. It contains the following possible values:
* 1 = Within the past 30 days
* 2 = More than 30 days ago but within the past 12 mos
* 3 = More than 12 months ago
* 8 = Misused at some point in past 12 mos LOG ASSN
* 9 = Misused at some point in lifetime LOG ASSN
* 11 = Used in the past 30 days LOGICALLY ASSIGNED
* 83 = DID NOT MISUSE PST 12 MOS (LIFETIME UNK) Log assn
* 91 = NEVER USED/MISUSED PAIN RELIEVERS
* 98 = BLANK (NO ANSWER)

And here's a corresponding question that answers if patients have used any pain relievers in the past 12 months:
PNRANYREC MOST RECENT ANY PAIN RELIEVER USE.  It comes from the pain reliever screener portion of the survey.
* 1 = Within the past 12 months
* 2 = More than 12 months ago
* 9 = Used at some point in the lifetime LOG ASSN
* 83 = DID NOT USE PST 12 MOS (LIFETIME UNK) Log assn
* 91 = NEVER USED PAIN RELIEVERS
* 98 = BLANK (NO ANSWER)

Let's first get counts of each variable.

In [0]:
#Breakdown of PNRNMREC (misuse)
dfNSDUHRaw.groupby(['PNRNMREC'])['QUESTID2'].count()

PNRNMREC
1       777
2      1906
3      3160
8        20
9        37
11        1
83      485
91    49676
98      214
Name: QUESTID2, dtype: int64

In [0]:
#Breakdown of PNRANYREC (any use)
dfNSDUHRaw.groupby(['PNRANYREC'])['QUESTID2'].count()

PNRANYREC
1     16768
2     13180
9       153
83      343
91    25647
98      185
Name: QUESTID2, dtype: int64

Combining the data from PNRNMREC and PNRANYREC, we can filter the data for patients who have used prescription painkillers in the past 12 months (PNRANYREC==1).

In [0]:
dfUsePainMeds = dfNSDUHRaw[dfNSDUHRaw.PNRANYREC==1]
dfUsePainMeds

,QUESTID2,FILEDATE,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,POVERTY3,TOOLONG,TROUBUND,PDEN10,COUTYP4,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP
19,11577143,10/09/2018,2,4,4,991,9991,91,91,91,...,3.0,2,2,1,1,2,2,54.413648,40017,1
20,34608143,10/09/2018,1,99,99,12,9999,99,1,30,...,2.0,2,2,2,2,2,2,3481.596088,40013,2
21,33708143,10/09/2018,1,99,99,17,9999,99,4,93,...,2.0,2,2,2,2,2,2,1022.448042,40042,2
23,58638143,10/09/2018,2,99,99,991,9991,91,91,91,...,3.0,2,2,1,1,2,2,628.886384,40016,2
33,25359143,10/09/2018,2,99,99,991,9991,91,91,91,...,2.0,2,2,2,2,2,2,839.107735,40039,1
34,17779143,10/09/2018,1,99,99,18,9999,99,2,93,...,3.0,2,2,2,2,2,2,1400.781545,40017,2
36,45789143,10/09/2018,1,99,99,18,9999,99,4,93,...,2.0,1,1,2,3,2,2,1144.049469,40044,1
46,46911143,10/09/2018,2,99,99,991,9991,91,91,91,...,1.0,2,2,2,2,2,2,3090.284207,40006,2
50,67571143,10/09/2018,2,99,99,991,9991,91,91,91,...,3.0,2,2,2,2,2,2,4215.062783,40040,2
52,76622143,10/09/2018,2,4,4,991,9991,91,91,91,...,3.0,2,2,1,1,2,2,4121.742730,40043,1


So now we have a table of people who have used painkillers in the past 12 months.

## 2.2 Set Dependent Variable

Here we'll name our dependent variable MISUSE and set it = 1 if PNRNMREC %in% {1, 2, or 8}, otherwise = 0

In [0]:
#Set MISUSE variable
misuseCodes = [1, 2, 8]
dfUsePainMeds['MISUSE'] = dfUsePainMeds.apply(lambda row:
                                              1 if row.PNRNMREC in misuseCodes else 0,
                                              axis=1)

dfUsePainMeds

C:\Users\camke\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,QUESTID2,FILEDATE,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,TOOLONG,TROUBUND,PDEN10,COUTYP4,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP,MISUSE
19,11577143,10/09/2018,2,4,4,991,9991,91,91,91,...,2,2,1,1,2,2,54.413648,40017,1,0
20,34608143,10/09/2018,1,99,99,12,9999,99,1,30,...,2,2,2,2,2,2,3481.596088,40013,2,0
21,33708143,10/09/2018,1,99,99,17,9999,99,4,93,...,2,2,2,2,2,2,1022.448042,40042,2,0
23,58638143,10/09/2018,2,99,99,991,9991,91,91,91,...,2,2,1,1,2,2,628.886384,40016,2,0
33,25359143,10/09/2018,2,99,99,991,9991,91,91,91,...,2,2,2,2,2,2,839.107735,40039,1,0
34,17779143,10/09/2018,1,99,99,18,9999,99,2,93,...,2,2,2,2,2,2,1400.781545,40017,2,1
36,45789143,10/09/2018,1,99,99,18,9999,99,4,93,...,1,1,2,3,2,2,1144.049469,40044,1,0
46,46911143,10/09/2018,2,99,99,991,9991,91,91,91,...,2,2,2,2,2,2,3090.284207,40006,2,1
50,67571143,10/09/2018,2,99,99,991,9991,91,91,91,...,2,2,2,2,2,2,4215.062783,40040,2,0
52,76622143,10/09/2018,2,4,4,991,9991,91,91,91,...,2,2,1,1,2,2,4121.742730,40043,1,0


In [0]:
#Determine classification imbalance
print('COUNT of MISUSE')
print(dfUsePainMeds.groupby(['MISUSE'])['MISUSE'].count())
print('\nPERCENT MISUSE')
print(sum(dfUsePainMeds.MISUSE) / len(dfUsePainMeds))

COUNT of MISUSE
MISUSE
0    14065
1     2703
Name: MISUSE, dtype: int64

PERCENT MISUSE
0.16119990458015268


16% is imbalanced but not horribly so, and the N sizes seem sufficient to make predictions.

## 2.2 Find and Remove Coluns with No Variation

Next we'll look for any columns that contain entirely the same set of data.  We'll first record them, and then we'll remove them.

In [0]:
#First get the column names before their removal
colsBefore = dfUsePainMeds.columns

#Remove columns with no variation
dfUsePainMeds = dfUsePainMeds.loc[:, (dfUsePainMeds != dfUsePainMeds.iloc[0]).any()]

#Then get remaining columns
colsAfter = dfUsePainMeds.columns

#Finally, take the difference to reveal which columns were removed
colsDiff = colsBefore.difference(colsAfter)

In [0]:
print('REMOVED COLUMNS:')
print(colsDiff)

REMOVED COLUMNS:
Index(['FILEDATE', 'IIPNRANYREC', 'IRPNRANYREC', 'PNRANYFLAG', 'PNRANYLIF',
       'PNRANYREC', 'PNRANYYR', 'PSYANYFLAG', 'PSYANYYR'],
      dtype='object')


## 2.3 Find and Remove Perfect Predictors

The trick here is to look through all the columns, and ensure that at least one value is in both classes of our dependent variable.

_Note, we must be sure not to remove the actual dependent variable, `MISUSE`, since it too will fit this description._

Here's how we'll tackle this problem.  We'll group counts by each variable and also by `MISUSE`. If the length of this groupby object is the same as the length of a groupby object with only the variable and not `MISUSE`, then we know there's no predictive information.  An illustration might be helpful.  First, let's look at the counts of records grouped by `HEALTH` and `MISUSE`.

In [0]:
variable = 'HEALTH'
print(dfUsePainMeds.groupby([variable, 'MISUSE'])[variable].count())

HEALTH  MISUSE
1       0         2577
        1          400
2       0         5080
        1         1001
3       0         4191
        1          881
4       0         1765
        1          353
5       0          447
        1           67
94      0            5
97      1            1
Name: HEALTH, dtype: int64


Here we see there are lots of records in multiple `HEALTH` responses for each of our two `MISUSE` classifications (0 and 1).

Now let's look at another example ...

In [0]:
#Example with no variation
variable = 'PNRNMREC'
print(dfUsePainMeds.groupby([variable, 'MISUSE'])[variable].count())

PNRNMREC  MISUSE
1         1           777
2         1          1906
3         0          1409
8         1            20
9         0            27
11        0             1
83        0           134
91        0         12473
98        0            21
Name: PNRNMREC, dtype: int64


Here we see with the `PNRNMREC` variable there are no responses that are in both categories of `MISUSE`.  But what we really want to notice is that the length of this variable _is the same regardless of whether it is grouped by `MISUSE`_.  So in these cases, when their length is the same, we can remove this variable since it will be a perfect predictor.

In [0]:
#Create empty list
colsToRemove = []

#Cycle through all columns, compare lengths, add to list if same length
# for i, col in enumerate(dfUsePainMeds.columns):
#     print(i, 'COLUMN NAME:', col)
#     if len(dfUsePainMeds.groupby([col, 'MISUSE'])) == len(dfUsePainMeds.groupby([col])):
#         print('FOUND ONE!')
#         colsToRemove.append(col)
'''Commented this out because it takes a long time (~1 hr) to run.
Horribly inefficient code...must be a better way. But it was a 1 time task and it's done.
See results below:
'''
        
colsToRemove = ['QUESTID2', 'PNRNMREC', 'PNRNM30D', 'MISUSE']

#Remove the `MISUSE` category from the list, which will always be the final category
colsToRemove = colsToRemove[:-1]

print('COLUMNS TO REMOVE:')
print(colsToRemove)

#Remove the columns
dfUsePainMeds = dfUsePainMeds.drop(colsToRemove, axis=1)

COLUMNS TO REMOVE:
['QUESTID2', 'PNRNMREC', 'PNRNM30D']


Unfortunately looking for perfect multicolinearity with our `MISUSE` outcome variable only resulted in the removal of 3 columns.

# Ideas

* Remove columns that are "downstream" from MISUSE (e.g., have you ever misused oxycontin) and would therefore perfectly predict the dependent variable.
* Need to recode the variables, potentially one-hot encode those that need it ... need a good way to determine this
* We could look for class balance of < 90%, to eliminate the near-perfect predictors.
* Need to whittle down to a few variables that we can ask on a questionnaire
* Rather than a top-down approach, perhaps we use a bottom-up approach, and build a model that uses only a handful of variables, just so we can get something built end-to-end.  Once complete, or in parallel, we could then improve our features and continuously feed them into new versions of the model.

# Doodling

In [0]:
variable = 'HEALTH'
print(dfUsePainMeds.groupby([variable])[variable].count())

HEALTH
1     2977
2     6081
3     5072
4     2118
5      514
94       5
97       1
Name: HEALTH, dtype: int64


In [0]:
dfUsePainMeds

,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,CIG30AV,...,TOOLONG,TROUBUND,PDEN10,COUTYP4,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP,MISUSE
19,2,4,4,991,9991,91,91,91,91,91,...,2,2,1,1,2,2,54.413648,40017,1,0
20,1,99,99,12,9999,99,1,30,99,4,...,2,2,2,2,2,2,3481.596088,40013,2,0
21,1,99,99,17,9999,99,4,93,93,93,...,2,2,2,2,2,2,1022.448042,40042,2,0
23,2,99,99,991,9991,91,91,91,91,91,...,2,2,1,1,2,2,628.886384,40016,2,0
33,2,99,99,991,9991,91,91,91,91,91,...,2,2,2,2,2,2,839.107735,40039,1,0
34,1,99,99,18,9999,99,2,93,93,93,...,2,2,2,2,2,2,1400.781545,40017,2,1
36,1,99,99,18,9999,99,4,93,93,93,...,1,1,2,3,2,2,1144.049469,40044,1,0
46,2,99,99,991,9991,91,91,91,91,91,...,2,2,2,2,2,2,3090.284207,40006,2,1
50,2,99,99,991,9991,91,91,91,91,91,...,2,2,2,2,2,2,4215.062783,40040,2,0
52,2,4,4,991,9991,91,91,91,91,91,...,2,2,1,1,2,2,4121.742730,40043,1,0
